<a href="https://colab.research.google.com/github/Yogs05/Task/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Prayoga Permana

NPM : 2206030621

Source : https://huggingface.co/alexgeh196/twitter-sentiment-analysis

In [1]:
!pip install transformers

In [2]:
!pip install torch

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer for the pretrained sentiment analysis model
tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")

# Load the pretrained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
def predict_sentiment_alternative(text):
    # Tokenize the input text and convert it to tensor format
    encoded_input = tokenizer.encode_plus(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    # Pass the inputs through the model to get the logits
    logits = model(**encoded_input).logits

    # Compute probabilities and determine the sentiment
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    sentiment_label = probabilities.argmax(dim=-1).item()

    # Map the sentiment label to a readable result
    sentiment_map = {0: "Negative", 1: "Positive"}
    return sentiment_map.get(sentiment_label, "Unknown")

# Example usage
text = "I love using deep learning models for sentiment analysis!"
print(predict_sentiment_alternative(text))


Positive


In [5]:
import pandas as pd

# Create a synthetic dataset
data = {
    "sentence": [
        "I love this product, it works amazingly well!",  # Positive
        "This was the worst experience of my life.",      # Negative
        "I would definitely recommend this to others!",   # Positive
        "Absolutely terrible service, never coming back.",# Negative
        "A fantastic outcome, exceeded all my expectations.", # Positive
        "So disappointing, not what I expected at all.",   # Negative
        "An excellent product, very satisfied!",           # Positive
        "Worst investment ever, a total waste of money.",  # Negative
        "Truly a remarkable and reliable product!",        # Positive
        "I regret buying this, nothing works as promised." # Negative
    ],
    "expected_sentiment": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

# Convert to DataFrame
synthetic_dataset = pd.DataFrame(data)

# Save to CSV
file_path = "/content/synthetic_sentiment_dataset.csv"
synthetic_dataset.to_csv(file_path, index=False)

# Provide download link
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# Test with a sample sentence

sample_sentence = "This product exceeded all my expectations!"
result = predict_sentiment_alternative(sample_sentence)
print(f"Sentiment of the sample sentence: {result}")

Sentiment of the sample sentence: Positive


In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the synthetic sentiment dataset
dataset = pd.read_csv("/content/synthetic_sentiment_dataset.csv")

# Initialize the pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")

# Define a function to predict the sentiment of a given text
def predict_sentiment(text):
    # Prepare the input for the model
    encoded_input = tokenizer(text, return_tensors="pt")

    # Get the model's predictions
    output = model(**encoded_input)

    # Compute probabilities and determine the sentiment label
    probabilities = torch.softmax(output.logits, dim=-1)
    predicted_label = torch.argmax(probabilities).item()

    return predicted_label  # 1 = Positive, 0 = Negative

# Evaluate the model's predictions against the expected sentiments
correct_count = 0

for _, row in dataset.iterrows():
    prediction = predict_sentiment(row['sentence'])
    if prediction == row['expected_sentiment']:
        correct_count += 1

# Calculate and display the model's accuracy
accuracy = (correct_count / len(dataset)) * 100
print(f"Accuracy of the Model on the Synthetic Dataset: {accuracy:.2f}%")


Accuracy of the Model on the Synthetic Dataset: 100.00%


In [10]:
import torch
import torch.nn as nn

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        if self.head_dim * num_heads != embed_size:
            raise ValueError("Embedding size must be divisible by the number of heads")

        # Linear layers for keys, queries, and values
        self.query_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.key_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.value_proj = nn.Linear(embed_size, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, queries, keys, values, mask=None):
        batch_size = queries.shape[0]

        # Project the input embeddings for keys, queries, and values
        queries = self.query_proj(queries).reshape(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        keys = self.key_proj(keys).reshape(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        values = self.value_proj(values).reshape(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Compute scaled dot-product attention
        scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.head_dim ** 0.5)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attention = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attention, values)

        # Concatenate heads and pass through final linear layer
        attention_output = attention_output.transpose(1, 2).reshape(batch_size, -1, self.embed_size)
        return self.fc_out(attention_output)

class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads, dropout_rate, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadSelfAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, queries, keys, values, mask=None):
        # Self-attention and residual connection
        attention_output = self.attention(queries, keys, values, mask)
        x = self.dropout(self.norm1(attention_output + queries))

        # Feed-forward network and residual connection
        feed_forward_output = self.feed_forward(x)
        out = self.dropout(self.norm2(feed_forward_output + x))
        return out


In [11]:
embed_size = 256  # Size of the embedding vectors
num_heads = 8  # Number of attention heads
dropout_rate = 0.1  # Dropout probability
feed_forward_expansion = 4  # Expansion factor for the feed-forward layer
attention_mask = None  # Optional mask for the attention mechanism

# Initialize the transformer block
transformer_layer = TransformerBlock(embed_size, num_heads, dropout_rate, feed_forward_expansion)

# Create dummy input tensor
input_tensor = torch.rand(32, 10, embed_size)  # Shape: (batch_size, sequence_length, embed_size)

# Pass the input through the transformer block
output_tensor = transformer_layer(input_tensor, input_tensor, input_tensor, attention_mask)
print(output_tensor.shape)  # Expected output: (32, 10, embed_size)


torch.Size([32, 10, 256])


In [12]:
# Test with a sample sentence
sample_sentence = "This product exceeded all my expectations!"
result = predict_sentiment(sample_sentence)
print(f"Sentiment of the sample sentence: {result}")

Sentiment of the sample sentence: 1


In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.metrics import accuracy_score

# Synthetic sentiment analysis dataset
data = {
    "sentence": [
        "I love this product, it works amazingly well!",  # Positive
        "This was the worst experience of my life.",      # Negative
        "I would definitely recommend this to others!",   # Positive
        "Absolutely terrible service, never coming back.",# Negative
        "A fantastic outcome, exceeded all my expectations.", # Positive
        "So disappointing, not what I expected at all.",   # Negative
        "An excellent product, very satisfied!",           # Positive
        "Worst investment ever, a total waste of money.",  # Negative
        "Truly a remarkable and reliable product!",        # Positive
        "I regret buying this, nothing works as promised." # Negative
    ],
    "expected_sentiment": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("alexgeh196/twitter-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("alexgeh196/twitter-sentiment-analysis")

# Sentiment prediction function
def get_sentiment(sentence):
    # Tokenize input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Get model output and apply softmax to get probabilities
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)

    # Get the predicted sentiment (1 = Positive, 0 = Negative)
    return torch.argmax(probs, dim=1).item()

# Apply prediction function to the dataset
df["predicted_sentiment"] = df["sentence"].map(get_sentiment)

# Calculate accuracy
model_accuracy = accuracy_score(df["expected_sentiment"], df["predicted_sentiment"])
print(f"Accuracy of the Sentiment Model on the Synthetic Dataset: {model_accuracy*100:.2f} percent")


Accuracy of the Sentiment Model on the Synthetic Dataset: 100.00 percent
